<a href="https://colab.research.google.com/github/Larissabraz/bootcamp-mia-2022/blob/main/8.%20Clustering/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering

Nesse notebook iremos explorar alguns dos modelos de clustering :D

### Tipos de clustering

- **Por partição**
- **Hierárquico**
- **Por densidade**

## Por partição: **K-means**

![](https://media.giphy.com/media/VryvUKuOxNLqM/giphy.gif)

### Exercício 1

In [ ]:
# imports necessários para a aula
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = (12, 7)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importar o dataset
df = pd.read_csv("https://github.com/WoMakersCode/data-science-bootcamp/raw/master/4.1%20Clustering/data/case.csv")

Vamos utilizar uma base fictícia contendo dados de visitas de clientes em um site que gostaríamos de segmentar:  
- **Visitas**: quantidade de visitas realizadas durante o mês
- **Tempo**: tempo, em segundos, que os usuários ficaram no site

In [ ]:
# TODO
# Ver a carinha do nosso conjunto de dados

**Vamos visualizar a distribuição desses dados?**

In [ ]:
plt.scatter(df.visitas, df.tempo, alpha=0.5)
plt.xlabel('Tempo')
plt.ylabel('Quantidade de visitas')
plt.show()

**IMPORTANTE**

Como os agrupamentos são definidos com base em uma medida de distância, primeiro **precisamos normalizar os dados**!

In [ ]:
# Importar o StandardScaler e normalizar os dados
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df),columns = df.columns)

In [ ]:
plt.scatter(df.visitas, df.tempo, alpha=0.5)
plt.xlabel('Tempo')
plt.ylabel('Quantidade de visitas')
plt.show()

**Voltando ao K-means...**



O Sklearn já conta com uma implementação do [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). Podemos importá-la:

In [ ]:
# Importar o K-means
from sklearn.cluster import KMeans

In [ ]:
# cria uma instância do K-means
kmeans = KMeans() 
kmeans.fit(df)
# salva os centroides
centroides = kmeans.cluster_centers_
# salva as labels dos clusters para cada exemplo
y_kmeans = kmeans.predict(df)

In [ ]:
# plota os dados identificando seus clusters
plt.scatter(df.visitas, df.tempo, c=y_kmeans, alpha=0.5, cmap='rainbow')
plt.xlabel('Tempo')
plt.ylabel('Quantidade de visitas')
# plota os centroides também
plt.scatter(centroides[:, 0], centroides[:, 1], c='black', marker='X', s=200, alpha=0.5)
plt.show()

In [ ]:
# TODO
# Rodar o K-means definindo o número de clusters como 50

Altere o número de clusters e rode o algoritmo de novo. Vamos ver o que acontece :D

Não se esqueça de adicionar uma seed!

In [ ]:
# TODO
# Rodar o K-means escolhendo o número de clusters que você acha que faz sentido

![](https://github.com/mulheres-em-ia/bootcamp-mia-2022/blob/main/8.%20Clustering/figures/inercia.png?raw=1)

Para escolhermos o número de clusters, observamos o gráfico do cotovelo com as inércias e escolhemos o ponto no qual a inércia começa a ficar mais plana e formar um "cotovelo":

In [ ]:
# Quantidade de clusters que serão testados
k = list(range(1, 10))

# Armazena das inércias para cada k
inercia = []

# Roda o K-means para cada k fornecido
for i in k:
    kmeans = KMeans(n_clusters=i, random_state=8)
    kmeans.fit(df)
    inercia.append(kmeans.inertia_)

# Plota o gráfico com as inércias
plt.plot(k, inercia, '-o')
plt.xlabel(r'Número de clusters')
plt.ylabel('Inércia')
plt.show()

### Exercício 2

Agora vamos fazer mais uma segmentação de clientes com o K-Means, dessa vez com mais features. O dataset que iremos utilizar é uma adaptação [deste aqui](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python) presente no Kaggle.

Dessa vez, vamos supor que estamos envolvidos em um projeto de um e-commerce que tem como objetivo segmentar e entender seus clientes para realizar campanhas de marketing.

In [ ]:
# importar o dataset
segmentation = pd.read_csv("https://github.com/WoMakersCode/data-science-bootcamp/raw/master/4.1%20Clustering/data/customer_segmentation.csv")

In [ ]:
segmentation.head()

Esse conjunto de dados possui 5 campos:

- **id**: código identificador do cliente
- **tem_cartao**: indica se o cliente tem cartão de crédito do e-commerce ou não
- **idade**: idade do cliente
- **renda**: renda mensal do cliente, em reais
- **score**: score indicando o gasto do cliente. Quanto maior, mais o cliente gasta no e-commerce

**Observando os dados acima, quais pré-processamentos vocês acham que serão necessários antes de realizarmos o agrupamento?**

**`1° - Remoção do identificador`**

O conjunto de dados contém o id do cliente que não iremos utilizar para a segmentação. Precisamos remover antes de realizar o agrupamento:

In [ ]:
segmentation.drop(columns='id', inplace=True)

**`2° - Lidar com feature categórica`**

Temos mais um ponto para resolver antes do agrupamento: a feature `tem_cartao` é categórica e o **k-means só lida com dados numéricos**.

![](http://giphygifs.s3.amazonaws.com/media/dJtDZzyjLF66I/giphy.gif)

**O que podemos fazer para lidar com variáveis categóricas então?**
- Feature engineering (One-hot enconding, Label Encoder, etc.)
- Utilizar outro algoritmo que permita usar esse tipo de variável

No nosso caso, vamos utilizar o [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html):

In [ ]:
# importar o LabelEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
segmentation['tem_cartao'] = label_encoder.fit_transform(segmentation.tem_cartao.values)

**`3° - Normalizar os dados`**

As escalas das features são diferentes, então precisamos normalizar os dados:

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_segmentation = pd.DataFrame(scaler.fit_transform(segmentation),columns = segmentation.columns)

**Agora sim podemos aplicar o K-means \o/**

Primeiro, vamos utilizar a regra do cotovelo para escolher o número de clusters:

In [ ]:
# TODO
# Fazer a curva do cotovelo aqui para escolhermos o número de clusters

Com base no gráfico acima, podemos escolher a quantidade de clusters que serão criados:

In [ ]:
# TODO
# Rodar o k-means no nosso conjunto de dados

Após o agrupamento, precisamos reverter a normalização para podermos interpretar os clusters formados!

In [ ]:
original_segmentation = pd.DataFrame(scaler.inverse_transform(scaled_segmentation),columns=segmentation.columns)

Como utilizamos 4 features para criação dos clusters, não podemos visualizá-las como no 1° exercício. Podemos utilizar o [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) para tentar interpretar os clusters:

In [ ]:
original_segmentation['cluster'] = clusters

In [ ]:
sns.pairplot(original_segmentation, hue = 'cluster');

## Hierárquico: **Agrupamento Hierárquico Aglomerativo**

![](https://media.giphy.com/media/pSNCWCEAsgrAs/giphy.gif)

### Exercício 3

Vamos utilizar o mesmo conjunto de dados utilizado no segundo exercício do K-means para realizar um agrupamento hierárquico aglomerativo. Para esse agrupamento, precisaremos importar o dendograma do [Scipy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html).

O sklearn também possui um [módulo](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) para realizar um agrupamento hierárquico aglomerativo, mas é complicado visualizar o dendograma com ele, então vamos ficar com o scipy mesmo.

In [ ]:
# importar os módulos dendogram e linkage
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
# TODO
# Usar o método linkage para fazer o agrupamento hierárquico
h_cluster = 

In [ ]:
# TODO
plt.title('Dendograma')
plt.xlabel('Exemplos')
plt.ylabel('Distância')
# Chamar o dendrograma aqui
plt.show()

**Vamos testar outras abordagens de agrupamentos e métricas de distância?**

![](https://media.giphy.com/media/12zV7u6Bh0vHpu/giphy.gif)

## Por densidade: **DBSCAN**

![](https://media.giphy.com/media/lCL2GQewp7fkk/giphy.gif)

### Exercício 4

Vamos utilizar novamente o conjunto do primeiro exercício com o DBSCAN, que vamos importar do [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html):

In [ ]:
# Importar o DBSCAN
from sklearn.cluster import DBSCAN

In [ ]:
# TODO
# Escolha um epsilon e um minPts
dbscan = 
# salvar os clusters atribuídos para cada exemplo
clusters = 

In [ ]:
# plota os clusters encontrados
plt.scatter(df.visitas, df.tempo, c=clusters, alpha=0.5, cmap='rainbow')
plt.xlabel('Tempo')
plt.ylabel('Quantidade de visitas')
plt.show()